In [9]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# Chargement du fichier CSV original
# Remplacez par votre chemin : "C:\\Users\\ZEJLI\\Projet 5A\\bdd_temperature\\df_clean.csv"
path_csv = "df_clean.csv" 
df_raw = pd.read_csv(path_csv, index_col=0)
df_raw.index = pd.to_datetime(df_raw.index)

# Création de df_model avec feature engineering
df_model = df_raw.copy()
df_model = df_model.sort_values(['latitude', 'longitude', 'time'])

# Features temporelles
df_model['month'] = df_model.index.month
df_model['month_sin'] = np.sin(2 * np.pi * df_model['month'] / 12)
df_model['month_cos'] = np.cos(2 * np.pi * df_model['month'] / 12)

# Index temporel pour le GP
min_date = df_model.index.min()
df_model['time_idx'] = (df_model.index.year - min_date.year) * 12 + (df_model.index.month - min_date.month)

print("✅ df_model créé avec succès.")

✅ df_model créé avec succès.


In [10]:
import os
import folium
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
from cartiflette import carti_download

# 1. Création de l'arborescence de dossiers
folders = ["cartes_climat/historique", "cartes_climat/predictions_2008"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# 2. Téléchargement et filtrage des 4 départements (Auvergne)
deps_codes = ['63', '03', '15', '43']
departements_fr = carti_download(
    values=["France"],
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

auvergne_deps = departements_fr[departements_fr['INSEE_DEP'].isin(deps_codes)].copy()
print(f"✅ Départements chargés : {', '.join(deps_codes)}")

✅ Départements chargés : 63, 03, 15, 43


In [11]:
# Conversion du DataFrame global en GeoDataFrame
gdf_total = gpd.GeoDataFrame(
    df_model, # Utilise le dataframe de l'étape précédente
    geometry=gpd.points_from_xy(df_model['longitude'], df_model['latitude']),
    crs="EPSG:4326"
)

# Jointure spatiale pour ne garder que les points des 4 départements
gdf_auvergne = gpd.sjoin(gdf_total, auvergne_deps[['INSEE_DEP', 'geometry']], predicate="within")

# Séparation Historique / Prédiction
df_hist = gdf_auvergne[gdf_auvergne.index.year <= 2007].copy()
# df_2008 doit contenir vos colonnes 'skt' (vérité) et 'pred_gp' ou 'pred_lstm'

In [12]:
# 1. Séparation temporelle (Train: 2000-2007, Test: 2008)
# On utilise gdf_auvergne qui contient les 4 départements
df_train = gdf_auvergne[gdf_auvergne.index.year <= 2007].copy()
df_test_raw = gdf_auvergne[gdf_auvergne.index.year == 2008].copy()

# 2. Normalisation (Obligatoire pour le bon fonctionnement du GP)
scaler_skt = StandardScaler()
scaler_geo = StandardScaler()

df_train['skt_norm'] = scaler_skt.fit_transform(df_train[['skt']])
df_train[['lat_norm', 'lon_norm']] = scaler_geo.fit_transform(df_train[['latitude', 'longitude']])

# On applique les réglages du train sur le test
df_test_raw['skt_norm'] = scaler_skt.transform(df_test_raw[['skt']])
df_test_raw[['lat_norm', 'lon_norm']] = scaler_geo.transform(df_test_raw[['latitude', 'longitude']])

print(f"✅ df_train et df_test_raw sont prêts ({len(df_train)} points d'entraînement).")

✅ df_train et df_test_raw sont prêts (28896 points d'entraînement).


In [13]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ExpSineSquared, ConstantKernel

# 1. Préparation des données et suppression des doublons exacts
# Si deux points ont la même position au même moment, le GP plante.
df_train_clean = df_train.drop_duplicates(subset=['time_idx', 'lat_norm', 'lon_norm'])

# 2. Échantillonnage
df_sample = df_train_clean.sample(n=min(len(df_train_clean), 1500), random_state=42)

# 3. Nouveau Kernel avec paramètres de base plus larges
kernel = (
    ConstantKernel(1.0, (1e-3, 1e3)) * ExpSineSquared(periodicity=12.0, length_scale=1.0) + 
    Matern(length_scale=1.0, nu=1.5) + 
    WhiteKernel(noise_level=0.1)
)

# --- LA CORRECTION EST ICI : alpha=1e-2 ---
# alpha ajoute une petite valeur sur la diagonale de la matrice pour la stabiliser
gp = GaussianProcessRegressor(
    kernel=kernel, 
    alpha=1e-2, 
    n_restarts_optimizer=3, 
    normalize_y=True
)

print("🚀 Entraînement du GP stabilisé...")
try:
    gp.fit(df_sample[['time_idx', 'lat_norm', 'lon_norm']].values, df_sample['skt_norm'].values)
    print("✅ Modèle entraîné avec succès !")
except Exception as e:
    print(f"❌ Nouvelle erreur : {e}")

# 4. Prédiction pour 2008
X_test = df_test_raw[['time_idx', 'lat_norm', 'lon_norm']].values
y_pred_norm = gp.predict(X_test)

df_2008 = df_test_raw.copy()
df_2008['pred_gp'] = scaler_skt.inverse_transform(y_pred_norm.reshape(-1, 1)).flatten()

🚀 Entraînement du GP stabilisé...


c:\Users\ZEJLI\anaconda3\envs\Transformers\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 0 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\ZEJLI\anaconda3\envs\Transformers\lib\site-packages\sklearn\gaussian_process\kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


✅ Modèle entraîné avec succès !


In [14]:
from scipy import stats
import numpy as np

# On filtre les valeurs aberrantes sur le réel et la prédiction
# On garde les données à l'intérieur de 3 écart-types
df_2008 = df_2008.copy()
df_2008['z_skt'] = np.abs(stats.zscore(df_2008['skt']))
df_2008['z_pred'] = np.abs(stats.zscore(df_2008['pred_gp']))
df_filtered_2008 = df_2008[(df_2008['z_skt'] < 3) & (df_2008['z_pred'] < 3)].copy()

print(f"✅ Filtrage Z-score terminé. Points conservés : {len(df_filtered_2008)}")

✅ Filtrage Z-score terminé. Points conservés : 3612


In [15]:
import folium
import branca.colormap as cm
from scipy import stats

def generate_pro_map(data, date_label, col_name, output_path, title_text):
    # 1. Fond de carte BLANC (Positron)
    m = folium.Map(location=[45.7, 3.2], zoom_start=8, tiles="CartoDB positron")
    
    # 2. CONVERSION DES BORNES POUR L'ÉCHELLE (Kelvin -> Celsius)
    # On soustrait 273.15 pour que l'échelle affiche des valeurs réelles en °C
    vmin_c = data[col_name].min() - 273.15
    vmax_c = data[col_name].max() - 273.15
    
    # Échelle Bleu -> Jaune -> Rouge (RdYlBu inversé)
    colors = cm.linear.RdYlBu_11.colors[::-1]
    colormap = cm.LinearColormap(colors=colors, vmin=vmin_c, vmax=vmax_c)
    colormap.caption = f"Température (°C)"
    colormap.add_to(m)

    # 3. Échantillonnage 1/2 pour la clarté
    sampled_data = data.iloc[::2]

    # 4. Contours des départements (Auvergne)
    folium.GeoJson(
        auvergne_deps,
        style_function=lambda x: {'fillColor': 'none', 'color': '#333', 'weight': 1.5, 'opacity': 0.4}
    ).add_to(m)

    # 5. Dessin des points INDIVIDUELS
    for _, row in sampled_data.iterrows():
        # --- CONVERSION DU POINT INDIVIDUEL ---
        val_k = row[col_name]
        val_c = val_k - 273.15  # Soustraction pour passer en Celsius
        
        # On utilise la valeur en Celsius pour obtenir la couleur correcte
        color_point = colormap(val_c)
        
        popup_html = f"""
        <div style="font-family: Arial; font-size: 12px; width: 160px;">
            <h4 style="margin:0; color:#333;">{title_text}</h4>
            <hr style="margin:5px 0;">
            <b>Mois :</b> {date_label}<br>
            <b>Température :</b> <b style="color:{color_point};">{val_c:.2f} °C</b><br>
            <b>Position :</b> {row['latitude']:.2f}, {row['longitude']:.2f}
        </div>
        """
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=color_point, # Contour du cercle
            fill=True,
            fill_color=color_point, # Remplissage du cercle
            fill_opacity=0.8,
            stroke=True,
            weight=0.8,
            popup=folium.Popup(popup_html, max_width=250)
        ).add_to(m)

    # 6. Titre de la carte
    title_html = f'''
             <div style="position: fixed; top: 10px; left: 50px; width: 300px; z-index:9999; 
                         background-color: white; border:2px solid black; padding: 10px; border-radius:5px;">
                 <b>{title_text}</b><br>Période : {date_label}
             </div>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(output_path)

In [16]:
# --- ÉTAPE 1 : Nettoyage Historique (2000-2007) ---
df_hist['z_score'] = np.abs(stats.zscore(df_hist['skt']))
df_hist_clean = df_hist[df_hist['z_score'] < 3].copy()
df_hist_clean['year_month'] = df_hist_clean.index.to_period('M')

print("⏳ Génération Historique...")
for period, group in df_hist_clean.groupby('year_month'):
    monthly = group.groupby(['latitude', 'longitude'])[['skt']].mean().reset_index()
    generate_pro_map(monthly, str(period), 'skt', f"cartes_climat/historique/hist_{period}.html", "Historique Auvergne")

# --- ÉTAPE 2 : Nettoyage et Production Prédiction (2008) ---
df_filtered_2008 = df_2008[(np.abs(stats.zscore(df_2008['skt'])) < 3)].copy()
df_filtered_2008['period'] = df_filtered_2008.index.to_period('M')

print("⏳ Génération Prédictions 2008...")
for p, group in df_filtered_2008.groupby('period'):
    monthly = group.groupby(['latitude', 'longitude'])[['skt', 'pred_gp']].mean().reset_index()
    
    # Carte PRÉDITE
    generate_pro_map(monthly, str(p), 'pred_gp', f"cartes_climat/predictions_2008/MAP_PRED_{p}.html", "Prédiction IA")
    # Carte RÉELLE
    generate_pro_map(monthly, str(p), 'skt', f"cartes_climat/predictions_2008/MAP_REEL_{p}.html", "Réalité Terrain")

print("✨ Terminé ! Vos cartes sont prêtes et variées en couleurs.")

⏳ Génération Historique...
⏳ Génération Prédictions 2008...
✨ Terminé ! Vos cartes sont prêtes et variées en couleurs.


In [17]:
from scipy import stats
import numpy as np

print("⏳ Nettoyage et génération des cartes historiques (2000-2007)...")

# 1. Nettoyage Z-score sur l'historique
# On enlève les valeurs aberrantes pour que l'échelle de couleurs soit propre
df_hist['z_score'] = np.abs(stats.zscore(df_hist['skt']))
df_hist_clean = df_hist[df_hist['z_score'] < 3].copy()

# 2. Création de la colonne de période (Mois)
df_hist_clean['year_month'] = df_hist_clean.index.to_period('M')

# 3. Boucle de génération
for period, group in df_hist_clean.groupby('year_month'):
    # Calcul de la moyenne spatiale mensuelle
    monthly_points = group.groupby(['latitude', 'longitude'])[['skt']].mean().reset_index()
    
    file_name = f"cartes_climat/historique/hist_{period}.html"
    
    # Appel de la nouvelle fonction "Pro"
    generate_pro_map(
        monthly_points, 
        str(period), 
        'skt', 
        file_name, 
        "Historique Auvergne (ERA5)"
    )
    
    # Optionnel : petit print pour suivre l'avancée car c'est long
    if str(period).endswith("-01"): 
        print(f"   -> Année {str(period)[:4]} en cours...")

print("✅ Cartes historiques terminées avec succès dans /historique")

⏳ Nettoyage et génération des cartes historiques (2000-2007)...
   -> Année 2000 en cours...
   -> Année 2001 en cours...
   -> Année 2002 en cours...
   -> Année 2003 en cours...
   -> Année 2004 en cours...
   -> Année 2005 en cours...
   -> Année 2006 en cours...
   -> Année 2007 en cours...
✅ Cartes historiques terminées avec succès dans /historique


In [18]:
# Préparation des périodes
df_filtered_2008['period'] = df_filtered_2008.index.to_period('M')

print("⏳ Génération des cartes en cours...")

for p, group in df_filtered_2008.groupby('period'):
    # Moyenne par point pour le mois
    monthly = group.groupby(['latitude', 'longitude'])[['skt', 'pred_gp']].mean().reset_index()
    
    # 1. Carte PRÉDITE
    path_pred = f"cartes_climat/predictions_2008/MAP_PRED_{p}.html"
    generate_pro_map(monthly, str(p), 'pred_gp', path_pred, "PRÉDICTION IA")
    
    # 2. Carte RÉELLE
    path_reel = f"cartes_climat/predictions_2008/MAP_REEL_{p}.html"
    generate_pro_map(monthly, str(p), 'skt', path_reel, "RÉALITÉ TERRAIN")

print(f"✅ Terminé ! Les fichiers sont dans /predictions_2008")

⏳ Génération des cartes en cours...
✅ Terminé ! Les fichiers sont dans /predictions_2008
